In [1]:
#from textblob import TextBlob
#from attributegetter import *
from generatengrams import ngrammatch
from gensim.models import Word2Vec
from Contexts import *
import json
from Intents import *
import random
import os
import re
import sqlite3

In [2]:
def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent
    for action in current_intent.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()
    '''
   #print("booked a cab")
    context = IntentComplete()
    #print('attributes-- ', attributes)
    if current_intent.action == 'FindRestaurants':
        res_list = FindRestaurants(attributes)
        result = "Fetched Restaurants :  \n"
        for rest in res_list:
            result = result  + 'Name : ' + rest[1] + ', Location : ' + rest[2] + ', Cusine : ' + rest[3] + ', Price : ' + rest[4] +'\n'
     
        return result , context
    else:
        data = allocateCab(current_intent, attributes, context)
        return data, context

def FindRestaurants(attributes):
    print("BOT: You requested for \'{}\' restaurants in the \'{}\' of city with price range \'{}\'".format(attributes['cusine'], attributes['neighbourhood'], attributes['price']))
    print("BOT: ", "Feteching results...")
    
    t = (attributes['neighbourhood'], attributes['cusine'], attributes['price'])
    database = "chatbot.db"
    conn = sqlite3.connect(database)
    c = conn.cursor()
    c.execute("SELECT * FROM restaurants WHERE neighbourhood=? AND cusine=? AND price=? COLLATE NOCASE", t)
    rest_list = c.fetchall()
    #print('---- List of Restaurants:----')
    #for rest in rest_list:
    #    print('Name : ' + rest[1] + ', Location : ' + rest[2] + ', Cusine : ' + rest[3] + ', Price : ' + rest[4])
        
    c.close()
    conn.close()
    return rest_list
    
def allocateCab(current_intent, attributes, context):
    #print(attributes)
    with open('results/results.txt') as json_file:
        json_data = json.load(json_file)
        return findCab(json_data, attributes)
        #print(json_data)
        
def findCab(json_data, attributes):
    #print(attributes)
    #filt_result = ""
    retdata = json_data['cabs']
    count = len(retdata);
    for record in retdata:
        count = count-1;
        #print(count)
        #print( record["passengers"])
        #print(attributes["passengers"])
        if attributes["vehicle"] in record["type"]  and count >= 1:
                if int(attributes["passengers"]) <= record["passengers"] and count >= 1:
                    #print(record["cab-number"])
                    rslt = 'Booked a ' + record["type"] + ' with Registration number: ' + str(record["cab-number"]) + " which costs Rs. " + str(record["price"])
                    return rslt
                elif count <= 0:
                    return "No Cabs Available with selected options"
        elif count <= 0:
            return "No Cabs Available with selected options"

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    
    for para in current_intent.params:
        if para.required:
            if para.name not in attributes:
                #Example of where the context is born, implemented in Contexts.py
                if para.name=='RegNo':
                    context = GetRegNo()
                #returning a random prompt frmo available choices.
                return random.choice(para.prompts), context

    return None, context


def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    #uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    #print ('attributes ', attributes)
    #print ('cleaned_inpt', cleaned_input)
    
    return attributes, cleaned_input

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    #print(clean_input)
    #Scoring Algorithm, can be changed.
    
    
    
    #choosing here the intent with the highest score
    if(current_intent==None):
        scores = ngrammatch(clean_input)
        scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    #print ('clean_input', clean_input)
    #print ('scores', scores)
    

    if(current_intent==None):
        #print('clean_input--  ', clean_input)
        if(clean_input=="search"):
            return loadIntent('params/newparams.cfg', 'SearchStore')
        #if(clean_input=='book'):
        #    return loadIntent('params/newparams.cfg','OrderBook')
        if(clean_input=='cab'):
            #print('clean_input', clean_input)
            return loadIntent('params/newparams.cfg','BookCab')
        else:
            return loadIntent('params/newparams.cfg',scores[-1][0])
    else:
        #If current intent is not none, stick with the ongoing intent
        return current_intent

def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    #Can use context to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:
        #Code can be optimised here, loading the same files each time suboptimal 
        files = os.listdir('./entities/')
        #print(files)
        entities = {}
        for fil in files:
            if ('.ipynb_checkpoints' not in fil):
                #print('fil  -- ', fil)
                lines = open('./entities/'+fil).readlines()
                #print('lines ---  ',lines)
                for i, line in enumerate(lines):
                    lines[i] = line[:-1]
                entities[fil[:-4]] = '|'.join(lines)
                #print('entitiesfil' , entities)
                #entities {'cabdestinations': 'Kapra|Dr AS Rao Nagar|Cherlapalli|Meerpet HB Colony|Mallapur|Nacharam|Chilukanagar|Habsiguda|Ramanthapur|Uppal|Nagole|Mansoorabad|Hayaat nagar|BN Reddy Nagar|Vanasthalipuram|Hasthinapuram|Champapet|Lingojiguda|Saroornagar|Rama Krishna Puram|Kothapet|Chaitanyapuri|Gaddiannaram|Sai
       

        #Extract entity and update it in attributes dict
        for entity in entities:
            #print ('entityii ',entity)
            data = entities[entity].split('|')
            for i in data:
                if i.lower() in uinput.lower():
                    attributes[entity] = i
                    #print(i)
        for entity in entities:
                #print ('entity-- ',entity)
                uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)
                #print('uinput', uinput)
        #Example of where the context is being used to do conditional branching.
        if context.name=='GetRegNo' and context.active:
            #print ('regattributes ', attributes)
            match = re.search('[0-9]+', uinput)
            if match:
                uinput = re.sub('[0-9]+', '$regno', uinput)
                attributes['RegNo'] = match.group()
                context.active = False
        #print ('getattributesuiput', uinput)
        #print ('getattributes_attributes',attributes)
        return attributes, uinput

In [5]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts
        
        #Contexts are flags which control dialogue flow, see Contexts.py        
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        
        reset_keys = ['cancel', 'exit', 'restart', 'give up', 'start', 'not interested', 'bye', 'no thanks', 'reset']
        
        pattern = re.compile('|'.join(reset_keys))
        checkifCancelling = user_input
        if pattern.search(checkifCancelling.lower()):
            # self.context.IntentComplete()
            #Resets the state after the Intent is complete        
            self.context.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
            return 'you chose to reset the earlier preferences/selection \n BOT: Hi! How may I assist you?'
        
            
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
        #print(self.attributes)
        #print(self.clean_input)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context= check_actions(self.current_intent, self.attributes, self.context)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

In [9]:
session = Session()

print ('BOT: Hi! How may I assist you? type reset at any point to start afresh')

while True:
    
    inp = input('User: ')
    print ('BOT:', session.reply(inp))

BOT: Hi! How may I assist you? type reset at any point to start afresh
User: my wife spoiled the dish
BOT: To which locality you want to go
User: reset
BOT: you chose to reset the earlier preferences/selection 
 BOT: Hi! How may I assist you?
User: biryai
BOT: To which locality you want to go
User: reset
BOT: you chose to reset the earlier preferences/selection 
 BOT: Hi! How may I assist you?
User: biryani
BOT: In which part of city, do you want to look up the restaurants?(East, West, South, North)
User: east
BOT: What is your favorite cusine?
User: Indian 
BOT: What is your price range? (Cheap, Medium, expensive)
User: expensive
BOT: You requested for 'Indian' restaurants in the 'east' of city with price range 'Expensive'
BOT:  Feteching results...
BOT: Fetched Restaurants :  
Name : Ohris, Location : east, Cusine : Indian, Price : expensive
Name : Minerva, Location : east, Cusine : Indian, Price : expensive
Name : Taj Banjara, Location : east, Cusine : Indian, Price : expensive
Name

KeyboardInterrupt: 